# 神经网络复习
1.关键词：全连接网络（线性变换）、偏置、权重
2.h =xW +b      矩阵计算
3.激活函数（非线性变换）—sigmoid函数
y=σ(x)= 1/（ 1 +exp(−x) ）    （呈S形曲线）   求导dy/dx=y*(1-y)
接收任意大小的实数，输出0～1的实数

4.层的类化和正向传播的实现
所有的层都有forward()方法和backward()方法;
所有的层都有params和grads实例变量

In [5]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.Params=[]
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out
        return out
    
    def backforward(self,dout):
        return dout*self.out*(1-self.out)

class Affine:
    def __init__(self,w,b):
        self.Params=[w,b]
        self.x=None
        self.grads=[np.zeros_like(w),np.zeros_like(b)]

    def forward(self,x):
        w,b=self.Params
        h=np.dot(x,w)+b
        self.x=x
        return h

    def backward(self,dout):
        w,b=self.Params
        x=self.x
        db=np.sum(dout,axis=0)
        dx=np.dot(dout,w.T)
        dw=np.dot(x.T,dout)

        self.grads[0][...]=dw
        self.grads[1][...]=db

        return dx

class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size):
        I,H,O=input_size,hidden_size,output_size

        w1=np.random.randn(I,H)
        b1=np.random.randn(H)
        w2=np.random.randn(H,O)
        b2=np.random.randn(O)

        self.layers=[
            Affine(w1,b1),
            Sigmoid(),
            Affine(w2,b2)
        ]

        self.Params=[]
        for layer in self.layers:
            self.Params+=layer.Params

    def predict(self,x):
        for layer in self.layers:
            x=layer.forward(x)

        return x
    
x=np.random.randn(10,2)
model=TwoLayerNet(2,3,4)
res=model.predict(x)
print(res)

[[-1.27727393  0.94250444  0.27045385 -1.04991069]
 [-1.50235614  0.33290592  0.37814271 -0.98939756]
 [-1.61001545  0.5739993   0.33462916 -1.28620585]
 [-1.26986084  1.46912613  0.28018922 -1.23760909]
 [-1.46356943  0.40602428  0.37242359 -0.97174934]
 [-1.43134909  0.64290422  0.33382039 -1.06771653]
 [-1.10950785  0.52350731  0.28363701 -0.63554865]
 [-1.50721441  0.3172681   0.37794267 -0.99012632]
 [-1.2179484   1.76240424  0.30559916 -1.25218361]
 [-1.29472107  0.41630812  0.35035111 -0.76894961]]


5.神经网络的学习（反向）
损失函数（交叉熵误差cross entropy error），交叉熵误差由神经网络输出的各类别的概率和监督标签求得
Softmax层求Softmax 函数的值，用Cross Entropy Error层求交叉熵误差。

Softmax层： yk = exp(sk)/ (i=1~n求和)exp(si)
Softmax 函数输出的各个元素是0.0～1.0的实数。另外，如果将这些元素全部加起来，则和为1。因此，Softmax的输出可以解释为概率。 

Cross Entropy Error层   L =− （k=1~n求和）tk*logyk 
tk是对应于第k个类别的监督标签（one hot）
L =−1/N \*(n=1~N) (k) tnk*log(ynk)      N笔数据求平均值

两层合并为Softmax with Loss层

# 导数和梯度
W和dL/dW 具有相同的形状,矩阵和其梯度具有相同形状!
## 链式法则
误差反向传播法的关键是链式法则
1.加法节点将上游传来的值乘以1，再将该梯度向下游传播。
2.乘法节点
3.分支节点/repeat节点
4.sum节点
5.MatMul节点

In [4]:
import numpy as np

#repeat节点
D,N=8,7
x=np.random.randn(1,D)
y=np.repeat(x,N,axis=0)     #指定axis，可以指定沿哪个轴复制。

dy=np.random.randn(N,D)     
dx=np.sum(dy,axis=0,keepdims=True)      #指定keepdims=True，可以维持二维数组的维数。
print(dx)

#sum节点
x=np.random.randn(N,D)
y=np.sum(x,axis=0,keepdims=True)

dy=np.random.randn(1,D)
dx=np.repeat(dy,N,axis=0)
print(dx)
#Sum节点和Repeat节点存在逆向关系

#matmul节点
class Matmul:
    def __init__(self,w):
        self.Params=[w]
        self.x=None
        self.grads=[np.zeros_like(w)]

    def forward(self,x):
        self.x=x
        w,=self.Params
        out = np.dot(x,w)
        
        return out
    
    def backward(self,dout):
        w,=self.Params
        x=self.x

        dw=np.dot(x.T,dout)
        dx=np.dot(dout,w.T)

        self.grads[0][...]=[dw] #[...]浅拷贝和深拷贝区别
        return dx
    

[[-0.77826601  1.30531275 -3.344384    1.2860984  -2.04657512  0.61686348
  -1.81078717  1.2327498 ]]
[[-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]
 [-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]
 [-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]
 [-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]
 [-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]
 [-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]
 [-0.40568363 -1.09701156  0.20293221  0.61140193 -0.36182797 -0.44542417
  -0.91539304 -0.26337051]]
